In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Raul G. Martinez
## PID: A12461871
## UCSD MAS DSE Cohort 6
## DSE 203 - Fall 2020
**------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# Instructions

## Assignment 2

Due Nov 1 by 11:59pm 

Points 30 

Submitting a file upload 

Available Oct 23 at 12am - Nov 1 at 11:59pm 10 days

Data: citeseer.csv and dblp.csv  (in Canvas/Files)


**Your task:**

Entity Resolution of the two tables using the two different methods shown in class (dedupe and Anhai's py_entitymatching)

Your submission should include the  python scripts (includes SQL scripts) + results


**Suggested Reference:**

Jupyter Notebook I've uploaded: Entity Matching Complete Workflow.ipynb (Canvas/Files/Hands on/EM)

Anhai's Entity Matching Guide: https://sites.google.com/site/anhaidgroup/projects/magellan/py_entitymatching (Links to an external site.)

dedupe Slides: Canvas/Files/Hands-on Session/dedupe

dedupe Examples (A how-to-use guide through example programs): https://github.com/dedupeio/dedupe-examples

 

**Notes:**

(1) You have to install all the dependencies first before you can successfully install Anhai's py_entitymatching package

(2) If you try running the dedupe examples from this repo: https://github.com/dedupeio/dedupe-examples, (Links to an external site.) you might run into various issues. Read the dedupe Slides (in Canvas) to troubleshoot these.

# Solution

## Method 1: Anahi's py_entitymatching

In [2]:
# display python version used
!python --version

Python 3.7.9


In [3]:
import py_entitymatching as em
import py_stringmatching as sm
import pandas as pd
import os, sys

## Prepare the dataset

In [4]:
## Read dataset
A = em.read_csv_metadata('../../Data for Assignments/citeseer.csv',low_memory=False) # setting the parameter low_memory to False  to speed up loading.
B = em.read_csv_metadata('../../Data for Assignments/dblp.csv', low_memory=False)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
A = A[:100000]
B = B[:100000]

In [6]:
len(A), len(B)

(100000, 100000)

In [7]:
# find number of null values
A.isnull().sum()

id                       0
title                    6
authors                  0
journal              99992
month               100000
year                 80697
publication_type     99991
dtype: int64

In [8]:
# find number of null values
B.isnull().sum()

id                       0
title                    0
authors                  0
journal             100000
month               100000
year                     0
publication_type         0
dtype: int64

In [9]:
A.head()

,id,title,authors,journal,month,year,publication_type
0,1,An Arithmetic Analogue of Bezouts Theorem,David Mckinnon,NaN,NaN,NaN,NaN
1,2,Thompsons Group F is Not Minimally Almost Convex,"James Belk, Kai-uwe Bux",NaN,NaN,2002.0,NaN
2,3,Cognitive Dimensions Tradeoffs in Tangible User Interface Design,"Darren Edge, Alan Blackwell",NaN,NaN,NaN,NaN
3,4,"ACTIVITY NOUNS, UNACCUSATIVITY, AND ARGUMENT MARKING IN YUKATEKAN SSILA meeting; Special Session...","J. Bohnemeyer, Max Planck, I. Introduction",NaN,NaN,2002.0,NaN
4,5,PS1-6 A6 ULTRASOUND-GUIDED HIFU NEUROLYSIS OF PERIPHERAL NERVES TO TREAT SPASTICITY AND,"J. L. Foley, J. W. Little, F. L. Starr Iii, C. Frantz",NaN,NaN,NaN,NaN


In [10]:
B.head()

,id,title,authors,journal,month,year,publication_type
0,1,"Klaus Tschira Stiftung gemeinntzige GmbH, KTS",Klaus Tschira,NaN,NaN,2012,www
1,2,The SGML/XML Web Page,Robin Cover,NaN,NaN,2006,www
2,3,The Future of Classic Data Administration: Objects + Databases + CASE,Arnon Rosenthal,NaN,NaN,1998,www
3,4,XML Query Data Model,"Mary F. Fernandez, Jonathan Robie",NaN,NaN,2001,www
4,5,The XML Query Algebra,"Peter Fankhauser, Mary F. Fernndez, Ashok Malhotra, Michael Rys, Jrme Simon, Philip Wadler",NaN,NaN,2001,www


In [11]:
# drop all nan columns
A = A.drop(columns=['journal','month','year','publication_type'])
B = B.drop(columns=['journal','month','year','publication_type'])

In [12]:
# Set 'id' as the keys to the input tables
em.set_key(A, 'id')
em.set_key(B, 'id')

True

In [13]:
A.head()

,id,title,authors
0,1,An Arithmetic Analogue of Bezouts Theorem,David Mckinnon
1,2,Thompsons Group F is Not Minimally Almost Convex,"James Belk, Kai-uwe Bux"
2,3,Cognitive Dimensions Tradeoffs in Tangible User Interface Design,"Darren Edge, Alan Blackwell"
3,4,"ACTIVITY NOUNS, UNACCUSATIVITY, AND ARGUMENT MARKING IN YUKATEKAN SSILA meeting; Special Session...","J. Bohnemeyer, Max Planck, I. Introduction"
4,5,PS1-6 A6 ULTRASOUND-GUIDED HIFU NEUROLYSIS OF PERIPHERAL NERVES TO TREAT SPASTICITY AND,"J. L. Foley, J. W. Little, F. L. Starr Iii, C. Frantz"


In [14]:
B.head()

,id,title,authors
0,1,"Klaus Tschira Stiftung gemeinntzige GmbH, KTS",Klaus Tschira
1,2,The SGML/XML Web Page,Robin Cover
2,3,The Future of Classic Data Administration: Objects + Databases + CASE,Arnon Rosenthal
3,4,XML Query Data Model,"Mary F. Fernandez, Jonathan Robie"
4,5,The XML Query Algebra,"Peter Fankhauser, Mary F. Fernndez, Ashok Malhotra, Michael Rys, Jrme Simon, Philip Wadler"


## Downsampling

In [15]:
# Downsample the datasets 
sample_A, sample_B = em.down_sample(A, B, size=100, y_param=1, show_progress=True, n_jobs=-1)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\sampler\down_sample.py:354: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sample_table_splits = pd.np.array_split(sample_table_b, n_jobs)


In [16]:
# Display the number of tuples in the sampled datasets
len(sample_A), len(sample_B)

(99, 100)

In [17]:
# Show the metadata of sample_A, sample_B
em.show_properties(sample_A)

id: 1641593909512
key: id


In [18]:
em.show_properties(sample_B)

id: 1641594146376
key: id


In [19]:
sample_A.isnull().sum()

id         0
title      0
authors    0
dtype: int64

In [20]:
sample_B.isnull().sum()

id         0
title      0
authors    0
dtype: int64

In [21]:
sample_A.head()

,id,title,authors
2576,2577,Towards Using Simulation to Evaluate Safety Policy for Systems of Systems,"Robert Alex, Martin Hall-may, Georgios Despotou, Tim Kelly"
34849,34850,Using the Castle Metaphor to Communicate Basic Concepts in Cybersecurity Education,"G. Markowsky, L. Markowsky"
48,49,Oral Presentation Halogen-Free Process for the Conversion of Carbon Dioxide to,"Urethanes Homogeneous Catalysis, Mahmut Abla, Jun-chul Choi, Toshiyasu Sakakura, Bunh Co Etoh, E..."
1595,1596,AN IDENTITY RELATED TO THE RIESZ TRANSFORMS ON THE HEISENBERG GROUP DER-CHEN CHANG AND JINGZHI TIE,"Dedicated Professor, Stephen Wainger"
6205,6206,"Parallel Distributed Processing Challenges the Strong Modularity Hypothesis, Not the Locality As...",David C. Plaut


In [22]:
sample_B.head()

,id,title,authors
18131,18132,An Experimental Study of Vietnamese Question Answering System.,"Vu Mai Tran, Vinh Duc Nguyen, Oanh Thi Tran, Uyen Thu Thi Pham, Thuy Quang Ha"
75332,75333,A new algorithm for third generation circuit simulators: the one-step relaxation method.,"B. Hennion, P. Senn, D. Coquelle"
71924,71925,High Level Modelling and Design For a Microthreaded Scheduler to Support Microgrids.,"Nabil Hasasneh, Ian Bell, Chris R. Jesshope"
71550,71551,Advanced Relay Nodes for Adaptive Network Services - Concept and Prototype Experiment.,"Masayoshi Shimamura, Takeshi Ikenaga, Masato Tsuru"
89063,89064,Spatio-temporal Information Systems in a Statistical Context.,"Leonardo Tininini, Mario Paolucci, Giuseppe Sindoni, Stefano De Francisci"


## Data Exploration

In [23]:
## Open OpenRefine Server

## Blocking

### Checking similarities

In [25]:
# ws_tok_set = sm.WhitespaceTokenizer(return_set=True)

In [26]:
# jac = sm.Jaccard()
# lev = sm.Levenshtein()

In [27]:
# x = 'Evaluating Probabilistic Queries over Imprecise Data'
# y = 'Evaluating probabilistic queries over imprecise data'

In [28]:
# jac.get_raw_score(ws_tok_set.tokenize(x.lower()), ws_tok_set.tokenize(y.lower()))

In [29]:
# lev.get_sim_score(x, y)

In [38]:
# use rule-based blocker
rb = em.RuleBasedBlocker()
rb.add_rule(['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.4'], feature_table)

'_rule_0'

In [39]:
C1 = rb.block_tables(sample_A, sample_B, l_output_attrs=['title', 'authors'], r_output_attrs=['title', 'authors'], show_progress=False)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_stringsimjoin\utils\converter.py:99: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if col_type == pd.np.object:
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_stringsimjoin\utils\validation.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if attr_type != pd.np.object:


In [40]:
len(C1)

1

In [41]:
C1.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors
0,0,98104,8015,Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, Liwu Chang, Paul Syverson",Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks.,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, LiWu Chang, Paul F. Syverson"


In [42]:
# # Instantiate overlap blocker object
# ob = em.OverlapBlocker()

# # Specify the tokenization to be 'word' level and set overlap_size to be 3.
# C2 = ob.block_tables(sample_A, sample_B, 'title', 'title', word_level=True, overlap_size=10 , 
#                     l_output_attrs=['title', 'authors'], 
#                     r_output_attrs=['title','authors'],
#                     show_progress=False)

In [43]:
# len(C2)

In [44]:
# # Display first 5 tuple pairs in the candidate set.
# C2.head()

In [45]:
def title_title_function(x,y):
    
    # extract authors column
    x_author = x['authors']
    y_author = y['authors']
    
    # combine all last names in a string separated by a space
    x_lastnames = ' '.join([i.split(' ')[-1] for i in x_author.split(',')])
    y_lastnames = ' '.join([i.split(' ')[-1] for i in y_author.split(',')])
    
    # convert all last names to lower case
    x_lastnames = str(x_lastnames).lower()
    y_lastnames = str(y_lastnames).lower()
        
    # exclude when similarity score is lower than threshold
    if jac.get_raw_score(ws_tok_set.tokenize(x_lastnames), ws_tok_set.tokenize(y_lastnames)) < 0.4:
        return True
    else:
        return False

# Apply black box blocker
ws_tok_set = sm.WhitespaceTokenizer(return_set=True)
jac = sm.Jaccard()

bb = em.BlackBoxBlocker()
bb.set_black_box_function(title_title_function)

In [46]:
C3 = bb.block_tables(sample_A, sample_B, l_output_attrs=['title', 'authors'], r_output_attrs=['title', 'authors'], show_progress=False)

In [47]:
len(C3)

8

In [48]:
C3.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors
0,0,65152,24079,Supporting Social Interaction: Role of Social Presence Supporting Social Interaction in Virtual ...,"Kathy Shen, Angela Yu, Mohamed Khalifa, Kathy Ning Shen, Mohamed Khalifa, Angela Yan Yu",Build Up Organizational Innovativeness: An Integrated View.,"Mohamed Khalifa, Angela Yan Yu, Kathy Ning Shen"
1,1,71336,92678,Carlsberg: A Distributed Execution Environment Providing Coherent Shared Memory and Integrated M...,"Povl T. Koch, Robert J. Fowler",Efficient Processing of Expressive Node-Selecting Queries on XML Data in Secondary Storage: A Tr...,Christoph Koch
2,2,46286,52452,MobiEyes: Distributed Processing of Continuously Moving Queries on Moving Objects in a Mobile Sy...,Ling Liu,A Process View Framework for Artifact-Centric Business Processes.,"Sira Yongchareon, Chengfei Liu"
3,3,27896,5045,Abstract,"O G. Lobo, Cludio F. Lima, O G. Lobo, Ualg Informatics, Cludio F. Lima, Ualg Informatics",Parameter Setting in Evolutionary Algorithms,"Fernando G. Lobo, Cludio F. Lima, Zbigniew Michalewicz"
4,4,73985,97886,MULTIPLE FEATURE INTEGRATION FOR WRITER VERIFICATION,"Sung-hyuk Cha, Sargur, N. Srihari",Nearest Neighbor Search Using Additive Binary Tree.,"Sung-Hyuk Cha, Sargur N. Srihari"


In [49]:
## Exercise1: how to add custom stopwords like "title"

In [50]:
## Exercise2: how to make multiple rules in the blocker?
## Exercise3: how to define a blackbox blocker using your own rules?

### Combine all blocker outputs

In [51]:
C = em.combine_blocker_outputs_via_union([C1, C3])

In [52]:
len(C)

8

In [53]:
C

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors
0,0,23352,2990,Evaluating WordNet-based Measures of Lexical Semantic Relatedness,Er Budanitsky Graeme Hirst,Semantic Distance Measures with Distributional Profiles of Coarse-Grained Concepts.,"Graeme Hirst, Saif Mohammad"
1,1,27896,5045,Abstract,"O G. Lobo, Cludio F. Lima, O G. Lobo, Ualg Informatics, Cludio F. Lima, Ualg Informatics",Parameter Setting in Evolutionary Algorithms,"Fernando G. Lobo, Cludio F. Lima, Zbigniew Michalewicz"
2,2,36207,4596,MODELING CROWD BEHAVIOR USING EMOTIONAL ANTS,"Soumya Banarjee, Crina Grosan, Ajith Abraham",Stock Market Modeling Using Genetic Programming Ensembles.,"Crina Grosan, Ajith Abraham"
3,3,46286,52452,MobiEyes: Distributed Processing of Continuously Moving Queries on Moving Objects in a Mobile Sy...,Ling Liu,A Process View Framework for Artifact-Centric Business Processes.,"Sira Yongchareon, Chengfei Liu"
4,4,65152,24079,Supporting Social Interaction: Role of Social Presence Supporting Social Interaction in Virtual ...,"Kathy Shen, Angela Yu, Mohamed Khalifa, Kathy Ning Shen, Mohamed Khalifa, Angela Yan Yu",Build Up Organizational Innovativeness: An Integrated View.,"Mohamed Khalifa, Angela Yan Yu, Kathy Ning Shen"
5,5,71336,92678,Carlsberg: A Distributed Execution Environment Providing Coherent Shared Memory and Integrated M...,"Povl T. Koch, Robert J. Fowler",Efficient Processing of Expressive Node-Selecting Queries on XML Data in Secondary Storage: A Tr...,Christoph Koch
6,6,73985,97886,MULTIPLE FEATURE INTEGRATION FOR WRITER VERIFICATION,"Sung-hyuk Cha, Sargur, N. Srihari",Nearest Neighbor Search Using Additive Binary Tree.,"Sung-Hyuk Cha, Sargur N. Srihari"
7,7,98104,8015,Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, Liwu Chang, Paul Syverson",Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks.,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, LiWu Chang, Paul F. Syverson"


## Generating Features for Manually¶

In [24]:
## Getting Attribute Types
atypes1 = em.get_attr_types(sample_A)
atypes2 = em.get_attr_types(sample_B)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\attributeutils.py:191: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if returned_type == bool or returned_type == pd.np.bool_:


In [25]:
atypes1['title'], atypes2['title'], atypes1['authors'], atypes2['authors']

('str_gt_10w', 'str_bt_5w_10w', 'str_gt_10w', 'str_bt_5w_10w')

In [26]:
atypes2['title'] = 'str_gt_10w'
atypes1['authors'] = 'str_bt_5w_10w'

In [27]:
atypes1['title'], atypes2['title'], atypes1['authors'], atypes2['authors']

('str_gt_10w', 'str_gt_10w', 'str_bt_5w_10w', 'str_bt_5w_10w')

In [28]:
## Getting Attribute Correspondences
block_c = em.get_attr_corres(sample_A, sample_B)
id(A), id(block_c['ltable']), id(B), id(block_c['rtable'])

(1641262680520, 1641593909512, 1641262679048, 1641594146376)

In [29]:
block_c['corres']

[('id', 'id'), ('title', 'title'), ('authors', 'authors')]

In [30]:
block_c.keys()

dict_keys(['corres', 'ltable', 'rtable'])

In [31]:
## Getting tokenizers
# for blocking
tok = em.get_tokenizers_for_blocking()
# for matching
# tok = em.get_tokenizers_for_matching()

In [32]:
tok

{'qgm_2': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram>,
 'qgm_3': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram>,
 'wspace': <function py_entitymatching.feature.tokenizers.tok_wspace>,
 'alphabetic': <function py_entitymatching.feature.tokenizers.tok_alphabetic>,
 'alphanumeric': <function py_entitymatching.feature.tokenizers.tok_alphanumeric>,
 'dlm_dc0': <function py_entitymatching.feature.tokenizers._make_tok_delim.<locals>.tok_delim>}

In [33]:
## Getting Similarity Functions
# for blocking
sim = em.get_sim_funs_for_blocking()

# for matching
# sim = em.get_sim_funs_for_matching()

In [34]:
sim

{'affine': <function py_entitymatching.feature.simfunctions.affine>,
 'hamming_dist': <function py_entitymatching.feature.simfunctions.hamming_dist>,
 'hamming_sim': <function py_entitymatching.feature.simfunctions.hamming_sim>,
 'lev_dist': <function py_entitymatching.feature.simfunctions.lev_dist>,
 'lev_sim': <function py_entitymatching.feature.simfunctions.lev_sim>,
 'jaro': <function py_entitymatching.feature.simfunctions.jaro>,
 'jaro_winkler': <function py_entitymatching.feature.simfunctions.jaro_winkler>,
 'needleman_wunsch': <function py_entitymatching.feature.simfunctions.needleman_wunsch>,
 'smith_waterman': <function py_entitymatching.feature.simfunctions.smith_waterman>,
 'overlap_coeff': <function py_entitymatching.feature.simfunctions.overlap_coeff>,
 'jaccard': <function py_entitymatching.feature.simfunctions.jaccard>,
 'dice': <function py_entitymatching.feature.simfunctions.dice>,
 'monge_elkan': <function py_entitymatching.feature.simfunctions.monge_elkan>,
 'cosine'

In [35]:
## Getting Features
feature_table = em.get_features(sample_A, sample_B, atypes1, atypes2, block_c, tok, sim)


In [36]:
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x0000017E22E034C8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x0000017E22E03438>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x0000017E231524C8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x0000017E231521F8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x0000017E23152288>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,title_title_cos_dlm_dc0_dlm_dc0,title,title,dlm_dc0,dlm_dc0,cosine,<function title_title_cos_dlm_dc0_dlm_dc0 at 0x0000017E23152168>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,authors_authors_jac_qgm_3_qgm_3,authors,authors,qgm_3,qgm_3,jaccard,<function authors_authors_jac_qgm_3_qgm_3 at 0x0000017E272F4048>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,authors_authors_cos_dlm_dc0_dlm_dc0,authors,authors,dlm_dc0,dlm_dc0,cosine,<function authors_authors_cos_dlm_dc0_dlm_dc0 at 0x0000017E272F4168>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,authors_authors_mel,authors,authors,None,None,monge_elkan,<function authors_authors_mel at 0x0000017E272F4288>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,authors_authors_lev_dist,authors,authors,None,None,lev_dist,<function authors_authors_lev_dist at 0x0000017E272F43A8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [37]:
#### Exercise: how to add a custom feature - to do Jaccard on title_authors and add it to feature table?

In [77]:
len(C)

8

In [78]:
# Sample  candidate set
S = em.sample_table(C, 6)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\sampler\single_table.py:103: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sample_indices = pd.np.random.choice(len(table), sample_size,


In [79]:
S

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors
0,0,23352,2990,Evaluating WordNet-based Measures of Lexical Semantic Relatedness,Er Budanitsky Graeme Hirst,Semantic Distance Measures with Distributional Profiles of Coarse-Grained Concepts.,"Graeme Hirst, Saif Mohammad"
1,1,27896,5045,Abstract,"O G. Lobo, Cludio F. Lima, O G. Lobo, Ualg Informatics, Cludio F. Lima, Ualg Informatics",Parameter Setting in Evolutionary Algorithms,"Fernando G. Lobo, Cludio F. Lima, Zbigniew Michalewicz"
2,2,36207,4596,MODELING CROWD BEHAVIOR USING EMOTIONAL ANTS,"Soumya Banarjee, Crina Grosan, Ajith Abraham",Stock Market Modeling Using Genetic Programming Ensembles.,"Crina Grosan, Ajith Abraham"
4,4,65152,24079,Supporting Social Interaction: Role of Social Presence Supporting Social Interaction in Virtual ...,"Kathy Shen, Angela Yu, Mohamed Khalifa, Kathy Ning Shen, Mohamed Khalifa, Angela Yan Yu",Build Up Organizational Innovativeness: An Integrated View.,"Mohamed Khalifa, Angela Yan Yu, Kathy Ning Shen"
5,5,71336,92678,Carlsberg: A Distributed Execution Environment Providing Coherent Shared Memory and Integrated M...,"Povl T. Koch, Robert J. Fowler",Efficient Processing of Expressive Node-Selecting Queries on XML Data in Secondary Storage: A Tr...,Christoph Koch
7,7,98104,8015,Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, Liwu Chang, Paul Syverson",Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks.,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, LiWu Chang, Paul F. Syverson"


In [80]:
# Label S
G = em.label_table(S, 'gold')

Column name (gold) is not present in dataframe


In [81]:
G

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold
0,0,23352,2990,Evaluating WordNet-based Measures of Lexical Semantic Relatedness,Er Budanitsky Graeme Hirst,Semantic Distance Measures with Distributional Profiles of Coarse-Grained Concepts.,"Graeme Hirst, Saif Mohammad",0
1,1,27896,5045,Abstract,"O G. Lobo, Cludio F. Lima, O G. Lobo, Ualg Informatics, Cludio F. Lima, Ualg Informatics",Parameter Setting in Evolutionary Algorithms,"Fernando G. Lobo, Cludio F. Lima, Zbigniew Michalewicz",0
2,2,36207,4596,MODELING CROWD BEHAVIOR USING EMOTIONAL ANTS,"Soumya Banarjee, Crina Grosan, Ajith Abraham",Stock Market Modeling Using Genetic Programming Ensembles.,"Crina Grosan, Ajith Abraham",0
4,4,65152,24079,Supporting Social Interaction: Role of Social Presence Supporting Social Interaction in Virtual ...,"Kathy Shen, Angela Yu, Mohamed Khalifa, Kathy Ning Shen, Mohamed Khalifa, Angela Yan Yu",Build Up Organizational Innovativeness: An Integrated View.,"Mohamed Khalifa, Angela Yan Yu, Kathy Ning Shen",0
5,5,71336,92678,Carlsberg: A Distributed Execution Environment Providing Coherent Shared Memory and Integrated M...,"Povl T. Koch, Robert J. Fowler",Efficient Processing of Expressive Node-Selecting Queries on XML Data in Secondary Storage: A Tr...,Christoph Koch,0
7,7,98104,8015,Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, Liwu Chang, Paul Syverson",Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks.,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, LiWu Chang, Paul F. Syverson",0


In [82]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcher\matcherutils.py:98: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  idx_values = pd.np.array(labeled_data.index.values)


In [83]:
I

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold
4,4,65152,24079,Supporting Social Interaction: Role of Social Presence Supporting Social Interaction in Virtual ...,"Kathy Shen, Angela Yu, Mohamed Khalifa, Kathy Ning Shen, Mohamed Khalifa, Angela Yan Yu",Build Up Organizational Innovativeness: An Integrated View.,"Mohamed Khalifa, Angela Yan Yu, Kathy Ning Shen",0
0,0,23352,2990,Evaluating WordNet-based Measures of Lexical Semantic Relatedness,Er Budanitsky Graeme Hirst,Semantic Distance Measures with Distributional Profiles of Coarse-Grained Concepts.,"Graeme Hirst, Saif Mohammad",0
5,5,71336,92678,Carlsberg: A Distributed Execution Environment Providing Coherent Shared Memory and Integrated M...,"Povl T. Koch, Robert J. Fowler",Efficient Processing of Expressive Node-Selecting Queries on XML Data in Secondary Storage: A Tr...,Christoph Koch,0


In [84]:
J

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold
7,7,98104,8015,Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, Liwu Chang, Paul Syverson",Distance Bounding Protocols: Authentication Logic Analysis and Collusion Attacks.,"Catherine Meadows, Radha Poovendran, Dusko Pavlovic, LiWu Chang, Paul F. Syverson",0
2,2,36207,4596,MODELING CROWD BEHAVIOR USING EMOTIONAL ANTS,"Soumya Banarjee, Crina Grosan, Ajith Abraham",Stock Market Modeling Using Genetic Programming Ensembles.,"Crina Grosan, Ajith Abraham",0
1,1,27896,5045,Abstract,"O G. Lobo, Cludio F. Lima, O G. Lobo, Ualg Informatics, Cludio F. Lima, Ualg Informatics",Parameter Setting in Evolutionary Algorithms,"Fernando G. Lobo, Cludio F. Lima, Zbigniew Michalewicz",0


In [85]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [86]:
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)


c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\attributeutils.py:191: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if returned_type == bool or returned_type == pd.np.bool_:


In [87]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='gold',
                            show_progress=False)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\extractfeatures.py:157: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  c_splits = pd.np.array_split(candset, n_procs)


AttributeError: module 'cloudpickle.cloudpickle' has no attribute 'dumps'

## Label the Sampled Set

In [149]:
# Label the sampled set
# Specify the name for the label column
G = em.label_table(C, 'gold_label')

Column name (gold_label) is not present in dataframe


In [150]:
len(G)

9

In [151]:
# Assume that we have labeled the data and stored it in 
# labeled_data_demo.csv

# path_labeled_data = datasets_dir + os.sep + 'labeled_data_demo.csv'
# G = em.read_csv_metadata(path_labeled_data, key='_id', 
#                          fk_ltable='ltable_id', fk_rtable='rtable_id',
#                          ltable=A, rtable=B)

In [152]:
G.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold_label
0,0,1569,76915,Statistical Performance Modeling and Optimization,"Xin Li, Jiayong Le, Lawrence T. Pileggi",STAC: statistical timing analysis with correlation.,"Jiayong Le, Xin Li, Lawrence T. Pileggi",0
1,1,13049,84477,Pacific Symposium on Biocomputing 14:264-275 (2009) DISSECTING THE INTERFACE BETWEEN SIGNALING A...,"Human B Cells, Kai Wang, Mariano J. Alvarez, Brygida C. Bisikirska, Rune Linding, Katia Basso, R...",Dissecting the Interface Between Signaling and Transcriptional Regulation in Human B Cells.,"Kai Wang, Mariano J. Alvarez, Brygida C. Bisikirska, Rune Linding, Katia Basso, Riccardo Dalla F...",0
2,2,29366,25032,Electric Reliability Technology Solutions and was funded by the Office of Electric,"Ian Dobson, Benjamin A. Carreras",Understanding the Effect of Risk Aversion on Risk.,"U. S. Bhatt, David E. Newman, Benjamin A. Carreras, Ian Dobson",0
3,3,31914,73030,Minimally Synchronous Parallel ML with Parallel Composition,"Radia Benheddi, Frdric Loulergue",Parallel composition and bulk synchronous parallel functional programming.,Frdric Loulergue,0
4,4,36852,45632,Seeing Speech: Capturing Vocal Tract Shaping Using Real-Time Magnetic Resonance Imaging Understa...,"Krishna Nayak, Dani Byrd, Shrikanth Narayanan",Accelerated 3D MRI of vocal tract shaping using compressed sensing and parallel imaging.,"Yoon-Chul Kim, Shrikanth S. Narayanan, Krishna S. Nayak",0


## Select the best learned matchers

In [67]:
# # Get the datasets directory
# datasets_dir = em.get_install_path() + os.sep + 'datasets'

# path_A = datasets_dir + os.sep + 'dblp_demo.csv'
# path_B = datasets_dir + os.sep + 'acm_demo.csv'
# path_labeled_data = datasets_dir + os.sep + 'labeled_data_demo.csv'

In [68]:
# A = em.read_csv_metadata(path_A, key='id')
# B = em.read_csv_metadata(path_B, key='id')
# # Load the pre-labeled data
# S = em.read_csv_metadata(path_labeled_data, 
#                          key='_id',
#                          ltable=A, rtable=B, 
#                          fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [153]:
# Split S into I an J
IJ = em.split_train_test(G, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcher\matcherutils.py:98: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  idx_values = pd.np.array(labeled_data.index.values)


In [114]:
I

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold_label
6,6,77629,32542,Code Size and Depth Flows in Genetic Programming,"Terence Soule, James A. Foster","Computational Complexity, Genetic Programming, and Implications.","Bart Rylander, Terence Soule, James A. Foster",0
3,3,31914,73030,Minimally Synchronous Parallel ML with Parallel Composition,"Radia Benheddi, Frdric Loulergue",Parallel composition and bulk synchronous parallel functional programming.,Frdric Loulergue,0
0,0,1569,76915,Statistical Performance Modeling and Optimization,"Xin Li, Jiayong Le, Lawrence T. Pileggi",STAC: statistical timing analysis with correlation.,"Jiayong Le, Xin Li, Lawrence T. Pileggi",1
5,5,47803,22785,An Analysis of Web Site Privacy Policy Evolution in the Presence of HIPAA,"Annie I. Antn, Julia B. Earp, Matthew W. Vail, Neha Jain, Carrie Gheen, Jack M. Frink",Privacy Policy Representation in Web-based Healthcare.,"Julia Brande Earp, Matthew W. Vail, Annie I. Antn",0


In [115]:
J

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold_label
7,7,79410,60717,1 PERILS AND PITFALLS OF PARALLEL DISCRETE-EVENT SIMULATION,Rajive L. Bagrodia,Transparent implementation of conservative algorithms in parallel simulation languages.,"Vikas Jha, Rajive L. Bagrodia",0
2,2,29366,25032,Electric Reliability Technology Solutions and was funded by the Office of Electric,"Ian Dobson, Benjamin A. Carreras",Understanding the Effect of Risk Aversion on Risk.,"U. S. Bhatt, David E. Newman, Benjamin A. Carreras, Ian Dobson",0
1,1,13049,84477,Pacific Symposium on Biocomputing 14:264-275 (2009) DISSECTING THE INTERFACE BETWEEN SIGNALING A...,"Human B Cells, Kai Wang, Mariano J. Alvarez, Brygida C. Bisikirska, Rune Linding, Katia Basso, R...",Dissecting the Interface Between Signaling and Transcriptional Regulation in Human B Cells.,"Kai Wang, Mariano J. Alvarez, Brygida C. Bisikirska, Rune Linding, Katia Basso, Riccardo Dalla F...",1
4,4,36852,45632,Seeing Speech: Capturing Vocal Tract Shaping Using Real-Time Magnetic Resonance Imaging Understa...,"Krishna Nayak, Dani Byrd, Shrikanth Narayanan",Accelerated 3D MRI of vocal tract shaping using compressed sensing and parallel imaging.,"Yoon-Chul Kim, Shrikanth S. Narayanan, Krishna S. Nayak",0
8,8,97482,98721,Digital Object Identifier (DOI) 10.1007/s00138-004-0162-5 Applications Layer-based video registr...,"Jiangjian Xiao, Mubarak Shah",Accurate Motion Layer Segmentation and Matting.,"Jiangjian Xiao, Mubarak Shah",0


In [116]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

In [117]:
# Generate a set of features
F = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\attributeutils.py:191: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if returned_type == bool or returned_type == pd.np.bool_:


In [118]:
F.feature_name

0                               id_id_exm
1                               id_id_anm
2                          id_id_lev_dist
3                           id_id_lev_sim
4             title_title_jac_qgm_3_qgm_3
5         title_title_cos_dlm_dc0_dlm_dc0
6                         title_title_mel
7                    title_title_lev_dist
8                     title_title_lev_sim
9         authors_authors_jac_qgm_3_qgm_3
10    authors_authors_cos_dlm_dc0_dlm_dc0
11                    authors_authors_mel
12               authors_authors_lev_dist
13                authors_authors_lev_sim
Name: feature_name, dtype: object

#### Extracting Feature Vector

In [119]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='label',
                            show_progress=False)

Column name (label) is not present in dataframe
The attributes mentioned in attrs_after is not present in the input table


AssertionError: The attributes mentioned in attrs_after is not present in the input table

In [75]:
# # Display first few rows
# H.head()

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,title_title_jac_qgm_3_qgm_3,...,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,label
430,430,l1494,r1257,4,0.20,0.466667,0.466667,0,0.000000,0.000000,...,0.000000,0.000000,0.445707,44.0,0.083333,1,1.0,0.0,1.0,0
35,35,l1385,r1160,4,0.20,0.466667,0.466667,0,0.000000,0.025641,...,0.000000,0.000000,0.589417,43.0,0.271186,1,1.0,0.0,1.0,0
394,394,l1345,r85,4,0.20,0.000000,0.000000,0,0.090909,1.000000,...,0.951111,0.945946,0.822080,172.0,0.338462,1,1.0,0.0,1.0,1
29,29,l611,r141,3,0.25,0.666667,0.666667,0,0.090909,0.049383,...,0.000000,0.000000,0.531543,26.0,0.277778,1,1.0,0.0,1.0,0
181,181,l1164,r1161,2,0.60,0.733333,0.733333,0,0.076923,1.000000,...,0.592593,0.668153,0.684700,34.0,0.244444,1,1.0,0.0,1.0,1


In [76]:
# # Check if the feature vectors contain missing values
# # A return value of True means that there are missing values
# any(pd.notnull(H))

True

In [77]:
# # Impute feature vectors with the mean of the column values.
# H = em.impute_table(H, 
#                 exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
#                 strategy='mean')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Selecting the best matcher using cross-validation

In [78]:
# # Select the best ML matcher using CV
# result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
#         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
#         k=5,
#         target_attr='label', metric_to_select_matcher='f1', random_state=0)
# result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.915322,0.950714,0.930980
1,RF,1.000000,0.950714,0.974131
2,SVM,0.977778,0.810632,0.883248
3,LinReg,1.000000,0.935330,0.966131
4,LogReg,0.985714,0.935330,0.958724


In [79]:
# result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x1435d2310>,5,0.95,1.000000,0.764706,0.933333,0.928571,0.915322
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1435f02d0>,5,1.00,1.000000,1.000000,1.000000,1.000000,1.000000
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x1435d23d0>,5,1.00,1.000000,0.888889,1.000000,1.000000,0.977778
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x143382e50>,5,1.00,1.000000,1.000000,1.000000,1.000000,1.000000
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x143384410>,5,1.00,0.928571,1.000000,1.000000,1.000000,0.985714


In [80]:
# result['drill_down_cv_stats']['recall']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x1435d2310>,5,0.95,1.000000,0.928571,0.8750,1.000000,0.950714
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1435f02d0>,5,0.95,1.000000,0.928571,0.8750,1.000000,0.950714
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x1435d23d0>,5,0.90,0.923077,0.571429,0.8125,0.846154,0.810632
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x143382e50>,5,0.95,1.000000,0.928571,0.8750,0.923077,0.935330
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x143384410>,5,0.95,1.000000,0.928571,0.8750,0.923077,0.935330


In [81]:
# result['drill_down_cv_stats']['f1']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x1435d2310>,5,0.950000,1.000000,0.838710,0.903226,0.962963,0.930980
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1435f02d0>,5,0.974359,1.000000,0.962963,0.933333,1.000000,0.974131
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x1435d23d0>,5,0.947368,0.960000,0.695652,0.896552,0.916667,0.883248
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x143382e50>,5,0.974359,1.000000,0.962963,0.933333,0.960000,0.966131
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x143384410>,5,0.974359,0.962963,0.962963,0.933333,0.960000,0.958724


In [82]:
### Exercise: how to add a feature to do Jaccard on title + authors and add it to the feature table?
## check if it improves the matcher

## Use a ML Matcher to Get Prediction and use Trigger to improve Results

In [120]:
# Split S into I an J
IJ = em.split_train_test(G, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcher\matcherutils.py:98: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  idx_values = pd.np.array(labeled_data.index.values)


In [121]:
len(I)

4

In [122]:
len(J)

5

In [123]:
I

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold_label
6,6,77629,32542,Code Size and Depth Flows in Genetic Programming,"Terence Soule, James A. Foster","Computational Complexity, Genetic Programming, and Implications.","Bart Rylander, Terence Soule, James A. Foster",0
3,3,31914,73030,Minimally Synchronous Parallel ML with Parallel Composition,"Radia Benheddi, Frdric Loulergue",Parallel composition and bulk synchronous parallel functional programming.,Frdric Loulergue,0
0,0,1569,76915,Statistical Performance Modeling and Optimization,"Xin Li, Jiayong Le, Lawrence T. Pileggi",STAC: statistical timing analysis with correlation.,"Jiayong Le, Xin Li, Lawrence T. Pileggi",1
5,5,47803,22785,An Analysis of Web Site Privacy Policy Evolution in the Presence of HIPAA,"Annie I. Antn, Julia B. Earp, Matthew W. Vail, Neha Jain, Carrie Gheen, Jack M. Frink",Privacy Policy Representation in Web-based Healthcare.,"Julia Brande Earp, Matthew W. Vail, Annie I. Antn",0


In [124]:
J

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,rtable_title,rtable_authors,gold_label
7,7,79410,60717,1 PERILS AND PITFALLS OF PARALLEL DISCRETE-EVENT SIMULATION,Rajive L. Bagrodia,Transparent implementation of conservative algorithms in parallel simulation languages.,"Vikas Jha, Rajive L. Bagrodia",0
2,2,29366,25032,Electric Reliability Technology Solutions and was funded by the Office of Electric,"Ian Dobson, Benjamin A. Carreras",Understanding the Effect of Risk Aversion on Risk.,"U. S. Bhatt, David E. Newman, Benjamin A. Carreras, Ian Dobson",0
1,1,13049,84477,Pacific Symposium on Biocomputing 14:264-275 (2009) DISSECTING THE INTERFACE BETWEEN SIGNALING A...,"Human B Cells, Kai Wang, Mariano J. Alvarez, Brygida C. Bisikirska, Rune Linding, Katia Basso, R...",Dissecting the Interface Between Signaling and Transcriptional Regulation in Human B Cells.,"Kai Wang, Mariano J. Alvarez, Brygida C. Bisikirska, Rune Linding, Katia Basso, Riccardo Dalla F...",1
4,4,36852,45632,Seeing Speech: Capturing Vocal Tract Shaping Using Real-Time Magnetic Resonance Imaging Understa...,"Krishna Nayak, Dani Byrd, Shrikanth Narayanan",Accelerated 3D MRI of vocal tract shaping using compressed sensing and parallel imaging.,"Yoon-Chul Kim, Shrikanth S. Narayanan, Krishna S. Nayak",0
8,8,97482,98721,Digital Object Identifier (DOI) 10.1007/s00138-004-0162-5 Applications Layer-based video registr...,"Jiangjian Xiao, Mubarak Shah",Accurate Motion Layer Segmentation and Matting.,"Jiangjian Xiao, Mubarak Shah",0


In [125]:
# Create a Decision Tree Matcher
dt = em.DTMatcher(name='DecisionTree', random_state=0)

In [126]:
# Generate a set of features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\attributeutils.py:191: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if returned_type == bool or returned_type == pd.np.bool_:


In [127]:
feature_table.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x000002C8462178B8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x000002C846217798>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x000002C8462163A8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x000002C8461E5EE8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x000002C8461E5D38>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [95]:
# # We will remove many of the features here to purposly create a poor model. This will make it easier 
# # to demonstrate triggers later
# F = feature_table.drop([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
# F

In [128]:
F = feature_table

In [83]:
F.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x000002C8473B3E58>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x000002C8375353A8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x000002C833EC2D38>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x000002C833EC2798>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x000002C8460EAA68>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [135]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='gold',
                            show_progress=False)

Column name (gold) is not present in dataframe
The attributes mentioned in attrs_after is not present in the input table


AssertionError: The attributes mentioned in attrs_after is not present in the input table

In [108]:
# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

NameError: name 'H' is not defined

In [93]:
# Fit the decision tree to the feature vectors
dt.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], target_attr='label')

In [94]:
# Use the decision tree matcher to predict if tuple pairs match
dt.predict(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], target_attr='predicted_labels', 
           return_probs=True, probs_attr='proba', append=True, inplace=True)
H.head()

,_id,ltable_id,rtable_id,id_id_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,label,predicted_labels,proba
430,430,l1494,r1257,4.0,0.083333,1.0,1.0,0.0,1.0,0,0,0.0
35,35,l1385,r1160,4.0,0.271186,1.0,1.0,0.0,1.0,0,0,0.0
394,394,l1345,r85,4.0,0.338462,1.0,1.0,0.0,1.0,1,1,1.0
29,29,l611,r141,3.0,0.277778,1.0,1.0,0.0,1.0,0,0,0.0
181,181,l1164,r1161,2.0,0.244444,1.0,1.0,0.0,1.0,1,1,1.0


#### Debug the ML Matcher

In [96]:
# Split H into P and Q
PQ = em.split_train_test(H, train_proportion=0.5, random_state=0)
P = PQ['train']
Q = PQ['test']

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/matcher/matcherutils.py:104: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  label_train = labeled_data.ix[idx_train]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/matcher/matcherutils.py:105: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  label_test = labeled_data.ix[idx_test]


In [97]:
# Debug RF matcher using GUI
em.vis_debug_dt(dt, P, Q, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        target_attr='label')

# We see with the debugger that the false negatives have completely different values in the Title attribute.
# This is most likly because we removed all of the features that compare the Title attribute from each table earlier. 


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/evaluation/evaluation.py:172: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  false_pos_ls = list(new_data_frame.ix[false_positive_indices].index.values)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/evaluation/evaluation.py:173: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  false_neg_ls = list(new_data_frame.ix[false_negative_indices].index.values)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/debugmatcher/debug_gui_utils.py:

In [98]:
# We can see which tuples are not predicted correctly
H[H['label'] != H['predicted_labels']]

,_id,ltable_id,rtable_id,id_id_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,label,predicted_labels,proba
371,371,l650,r1594,4.0,0.120000,1.0,1.0,0.0,1.0,1,0,0.500000
259,259,l938,r1090,5.0,0.200000,1.0,1.0,0.0,1.0,1,0,0.333333
346,346,l1681,r693,4.0,0.238095,1.0,1.0,0.0,1.0,1,0,0.500000
184,184,l891,r485,4.0,0.137931,1.0,1.0,0.0,1.0,1,0,0.500000
11,11,l1189,r1674,4.0,0.222222,1.0,1.0,0.0,1.0,1,0,0.250000
121,121,l169,r521,4.0,0.153846,1.0,1.0,0.0,1.0,1,0,0.500000
267,267,l120,r1181,4.0,0.216667,1.0,1.0,0.0,1.0,1,0,0.500000
147,147,l867,r1263,4.0,0.142857,1.0,1.0,0.0,1.0,1,0,0.333333


In [ ]:
#### Use trigger to improve result

In [99]:
# Use the constructor to create a trigger
mt = em.MatchTrigger()

In [100]:
# Add two rules to the rule-based matcher

# Since we removed all of the features comparing Title earlier, we want to now add a rule that compares Titles
mt.add_cond_rule(['title_title_lev_sim(ltuple, rtuple) > 0.7'], feature_table)
# The rule has two predicates, one comparing the titles and the other looking for an exact match of the years
mt.add_cond_rule(['title_title_lev_sim(ltuple, rtuple) > 0.4', 'year_year_exm(ltuple, rtuple) == 1'], feature_table)
mt.get_rule_names()

odict_keys(['_rule_0', '_rule_1'])

In [101]:
# Since we are using the trigger to fix a problem related to false negatives, we want the condition to be 
# True and the action to be 1. This way, the trigger will set a prediction to 1 when the rule returns True.

mt.add_cond_status(True)
mt.add_action(1)

True

In [102]:
## Use the Trigger to improve results
preds = mt.execute(input_table=H, label_column='predicted_labels', inplace=False)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/utils/generic_helper.py:228: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return (ft_df.ix[feature_name]['is_auto_generated'],
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/utils/generic_helper.py:229: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  ft_df.ix[feature_name]['simfunction'],
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/py_entitymatching/utils/generic_helper.py:230: FutureWarning: 
.ix is deprecated. Please use
.loc for label base

In [103]:
# We were able to significantly reduce the number of incorrectly labeled tuple pairs
preds[preds['label'] != preds['predicted_labels']]

,_id,ltable_id,rtable_id,id_id_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,label,predicted_labels,proba
11,11,l1189,r1674,4.0,0.222222,1.0,1.0,0.0,1.0,1,0,0.25
86,86,l1570,r516,4.0,0.223684,1.0,1.0,0.0,1.0,0,1,0.00
267,267,l120,r1181,4.0,0.216667,1.0,1.0,0.0,1.0,1,0,0.50


In [104]:
# We can see that the two tuples that are still labeled incorrectly are due to the title and authors being in the
# wrong column for one of the tuples.
pd.concat([S[S['_id'] == 11], S[S['_id'] == 267]])

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year,label
11,11,l1189,r1674,"Weimin Du, Xiangning Liu, Abdelsalam Helal",Multiview Access Protocols for Large-Scale Replication,1998,Multiview access protocols for large-scale replication,"Xiangning Liu, Abdelsalam Helal, Weimin Du",1998,1
267,267,l120,r1181,"w. Bruce kroft, James callan, erik w. Brown",fast incrremental indexiing for fulltext informtion retreval,1994,Fast Incremental Indexing For Full-Text Information Retrieval,"Eric W. Brown, James P. Callan, W. Bruce Croft",1994,1
